# ESDL Project: Modelling biomass with Deep Learning from land use and ESDL variables
## ideas
* linking land use layers with ESDL variables for modelling biomass
* training and testing neuronal network for estimating above ground biomass (reference data GlobBiomass 2010 is provided)
* testing for different world regions and different variable ensembles

### loading modules/packages

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from numpy import datetime64
from ipywidgets import interact 
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from dask.diagnostics import ProgressBar
import matplotlib.cm as cm

In [ ]:
from __future__ import absolute_import, division, print_function
from sklearn.metrics import r2_score #various classification, regression and clustering algorithms. We use for metrics
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns # data visualization library based on matplotlib
import tensorflow as tf #dataflow and differentiable programming (machine learning applications)
from tensorflow import keras #neural-network building blocks
from tensorflow.keras import layers
from tensorflow.keras.models import model_from_json
import os #operation system interface 
print(tf.__version__)

## list of variables
evaporation, potential_evaporation, bare_soil_evaporation, root_moisture, surface_moisture,
land_surface_temperature, soil_moisture, white_sky_albedo, black_sky_albedo, net_ecosystem_exchange, terrestrial_ecosystem_respiration, gross_primary_productivity, precipitation

In [ ]:
varlist = ["evaporation", "bare_soil_evaporation", "surface_moisture",
           "land_surface_temperature", "root_moisture", "black_sky_albedo", 
           "net_ecosystem_exchange", "terrestrial_ecosystem_respiration", 
           "gross_primary_productivity", "leaf_area_index", "precipitation"]

## Open the cube

In [ ]:
ESDC_img = xr.open_zarr("/home/jovyan/work/datacube/ESDCv2.0.0/esdc-8d-0.083deg-1x2160x4320-2.0.0.zarr")
ESDC_time = xr.open_zarr("/home/jovyan/work/datacube/ESDCv2.0.0/esdc-8d-0.083deg-184x270x270-2.0.0.zarr")

## Built yearly mean of 2010 for all selected variables 

build subset of data cube for year 2010 for Europe

In [ ]:
# define world region for biomass modelling
# Europe: lat 70, 30, lon -20, 35
# Asia: Lat 70, 5 Lon 35, 150
# Africa: Lat 30,-35 Lon -20, 50
# N-C-America: Lat 70,10 Lon -150,-60
# S-America: Lat 10,-55 Lon -90,-35
# SE-Asia-Aus: Lat 10,-40 Lon 95, 180

region = "Europe"
lat1 = 70
lat2 = 30
lon1 = -20
lon2 = 35

In [ ]:
ESDC2010 = ESDC_time.sel(time = slice('2010-01-01','2010-12-31'), lat = slice(lat1,lat2), lon = slice(lon1,lon2))

plot mean for selected variable

In [ ]:
for var in varlist:
    print(var)
    exec(var+"map" + "= ESDC2010." + var + ".mean(dim='time')")
    #exec(var+"map" + ".plot.pcolormesh(figsize=(16,10))")
    #plt.savefig(var + "_mean_2010_map.png",dpi = 300)
    #plt.show()

## open additional files 
biomass map 

In [ ]:
biomass_2010 = xr.open_rasterio("/home/jovyan/work/workspace/GlobBiomass_2010/GlobBiomass_2010_agb_0.083_avg_float.tif")

In [ ]:
biomass_2010 = biomass_2010.sel(y = slice(lat1,lat2), x = slice(lon1,lon2))

In [ ]:
biomass_2010[0,:,:].plot.pcolormesh(figsize=(16,10))
plt.title("Reference AGB 2010")
plt.savefig("biomass_2010_map_" + region + ".png",dpi = 300)
plt.show()

### load country map

In [ ]:
countries = xr.open_rasterio("/home/jovyan/work/workspace/Country_mask_2015/country_map_2015_0.0833_mod.tif")
countries = countries.sel(y = slice(lat1,lat2), x = slice(lon1,lon2))

### load land use fractions from land use change reconstruction HILDA+

In [ ]:
urban_frac_2010 = xr.open_rasterio("/home/jovyan/work/workspace/HILDA+_LU_2010/hilda_plus_2010_class1_frac_0.083_avg.tif")
crop_frac_2010 = xr.open_rasterio("/home/jovyan/work/workspace/HILDA+_LU_2010/hilda_plus_2010_class2_frac_0.083_avg.tif")
pasture_frac_2010 = xr.open_rasterio("/home/jovyan/work/workspace/HILDA+_LU_2010/hilda_plus_2010_class3_frac_0.083_avg.tif")
forest_frac_2010 = xr.open_rasterio("/home/jovyan/work/workspace/HILDA+_LU_2010/hilda_plus_2010_class4_frac_0.083_avg.tif")
shrub_frac_2010 = xr.open_rasterio("/home/jovyan/work/workspace/HILDA+_LU_2010/hilda_plus_2010_class5_frac_0.083_avg.tif")
other_frac_2010 = xr.open_rasterio("/home/jovyan/work/workspace/HILDA+_LU_2010/hilda_plus_2010_class6_frac_0.083_avg.tif")

create subsets

In [ ]:
urban_frac_2010 = urban_frac_2010.sel(y = slice(lat1,lat2), x = slice(lon1,lon2))
crop_frac_2010 = crop_frac_2010.sel(y = slice(lat1,lat2), x = slice(lon1,lon2))
pasture_frac_2010 = pasture_frac_2010.sel(y = slice(lat1,lat2), x = slice(lon1,lon2))
forest_frac_2010 = forest_frac_2010.sel(y = slice(lat1,lat2), x = slice(lon1,lon2))
shrub_frac_2010 = shrub_frac_2010.sel(y = slice(lat1,lat2), x = slice(lon1,lon2))
other_frac_2010 = other_frac_2010.sel(y = slice(lat1,lat2), x = slice(lon1,lon2))

## prepare country map for coordinate selection on land

In [ ]:
land_use_sum = urban_frac_2010 + crop_frac_2010 + pasture_frac_2010 + forest_frac_2010 + shrub_frac_2010 + other_frac_2010
land_coord = ((land_use_sum < 2) & (land_use_sum > 0))
land_mask = xr.ones_like(crop_frac_2010)
land_mask = land_mask.where((land_use_sum < 2) & (land_use_sum > 0), 0)
land_mask = land_mask[0,:,:]

In [ ]:
land_mask.plot.pcolormesh(figsize=(16,10))
plt.title("land mask")

### extract all land pixels for parameter table

In [ ]:
pos = np.array(land_mask != 0).flatten()
lat = np.array(np.repeat(countries.y, countries.shape[2]))[pos]
lon = np.array(np.repeat(countries.x, countries.shape[1]))[pos]

In [ ]:
pos = np.array(land_mask) != 0

In [ ]:
biomass_val = np.array(biomass_2010[0,:,:])
biomass_val = biomass_val[pos]

In [ ]:
lst = np.array(land_surface_temperaturemap)[pos]
prec = np.array(precipitationmap)[pos]
soilevap = np.array(bare_soil_evaporationmap)[pos]
albedo = np.array(black_sky_albedomap)[pos]
lai = np.array(leaf_area_indexmap)[pos]
evap = np.array(evaporationmap)[pos]
gpp = np.array(gross_primary_productivitymap)[pos]
nee = np.array(net_ecosystem_exchangemap)[pos]
rootm = np.array(root_moisturemap)[pos]
surfm = np.array(surface_moisturemap)[pos]
resp = np.array(terrestrial_ecosystem_respirationmap)[pos]
urban = np.array(urban_frac_2010[0,:,:])[pos]
crop = np.array(crop_frac_2010[0,:,:])[pos]
past = np.array(pasture_frac_2010[0,:,:])[pos]
forest = np.array(forest_frac_2010[0,:,:])[pos]
shrub = np.array(shrub_frac_2010[0,:,:])[pos]
other = np.array(other_frac_2010[0,:,:])[pos]

## setting up a pandas table for the descriptors and response variables
chose different variable ensembles and name the descriptor setting accordingly

In [ ]:
dataset0 = pd.DataFrame({"Lat":lat, "Lon":lon, "lst":lst, "prec": prec, "soilevap": soilevap, 
            "albedo":albedo, "lai":lai, "evap":evap, "gpp":gpp, "nee": nee, "rootm":rootm, "surfm":surfm, 
             "resp":resp, "LU_urban": urban, "LU_crop": crop, "LU_past": past, "LU_forest": forest, "LU_shrub": shrub, "LU_other": other, "agb":biomass_val})

#dataset0 = pd.DataFrame({"Lat":lat, "Lon":lon, "lst":lst, "prec": prec, "soilevap": soilevap, 
#            "albedo":albedo, "lai":lai, "evap":evap, "gpp":gpp, "nee": nee, "rootm":rootm, "surfm":surfm, 
#             "resp":resp, "agb":biomass_val})

#dataset0 = pd.DataFrame({"Lat":lat, "Lon":lon, "LU_urban": urban, "LU_crop": crop, "LU_past": past, "LU_forest": forest, "LU_shrub": shrub, "LU_other": other, "agb":biomass_val})

name = "all-var"

In [ ]:
dataset0.head()

In [ ]:
dataset = dataset0.dropna()
dataset = dataset.astype('float64')

In [ ]:
dataset_orig = dataset #keep a backup of the original dataset. Might be useful.
dataset.to_csv('dataset_clean_' + region + '_' + name + '.csv',index=False) #Saving the csv file just for easier visualization of the raw data

In [ ]:
sc = MinMaxScaler(feature_range = (0,1)) #Scaling features to a range between 0 and 1

In [ ]:
# Scaling and translating each feature to our chosen range
dataset = sc.fit_transform(dataset) 
dataset = pd.DataFrame(dataset, columns = dataset_orig.columns)
dataset_scaled = dataset #Just backup
inverse_data = sc.inverse_transform(dataset) #just to make sure it works

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0) # build training data set (random state is fixing the seed)
test_dataset = dataset.drop(train_dataset.index) #  take the rest as test data set
train_dataset_orig = dataset_orig.sample(frac=0.8,random_state=0) #just backup
test_dataset_orig =  dataset_orig.drop(train_dataset_orig.index) #just backup

In [ ]:
#Inspect the original mean
sns.set()
f, (ax1,ax2) = plt.subplots(2, 1,sharex=True)
sns.distplot(train_dataset["agb"],hist=True,kde=False,bins=75,color='darkblue',  ax=ax1, axlabel=False)
sns.kdeplot(train_dataset["agb"],bw=0.15,legend=True,color='darkblue', ax=ax2)

ax1.set_title('Original  histogram')
ax1.legend(['AGB'])
ax2.set_title('KDE')
ax2.set_xlabel('AGB')
ax1.set_ylabel('Count')
ax2.set_ylabel('Dist')
plt.savefig('histograms_' + region + '.png', bbox_inches='tight')
plt.show()

In [ ]:
#Check the overall stats
train_stats = train_dataset.describe()
train_stats.pop('agb') #because that is what we are trying to predict
train_stats = train_stats.transpose()
train_stats 

In [ ]:
# Remove the output from our list of predictors, save training dataset
train_dataset.to_csv('train_dataset_' + region + '_' + name + '.csv',index=False) 
train_labels = train_dataset.pop('agb')
#train_dataset.tail()

In [ ]:
# save testing dataset
test_dataset.to_csv('test_dataset_' + region + '_' + name + '.csv',index=False)
test_labels = test_dataset.pop('agb')
#test_dataset.tail()

## build the model

## setting up the neural network
### some variables for the optimizer SGD

In [ ]:
lr_val = 0.01 # change the learning rate to adapt optimizing behaviour, learning rate is a hyperparameter / how fast should it learn?
# small LR -> long convergence time
# large LR -> convergence problems
momentum_val = 0  # change the momentum to adapt optimizing behaviour
nesterov_val = 'True'
#value adapts the optimizing behaviour (rebalancing the direction between momentum and learning rate)
decay_val = 1e-6

In [ ]:
def build_model():
  model = keras.Sequential([
    ## how much layers? 
    # why 64 units? increase in number of units in hidden layers increases the chances of overfitting
    # activation function: rectified linear unit (relu)
    # insert layers of dropout in between (regularization to avoid overfitting)
    # how deep should the NN be? overfitting chances are increased!?
    
    layers.Dense(64,kernel_initializer='normal',activation=tf.nn.relu,input_shape=[len(train_dataset.keys())]),
    layers.Dropout(0.2),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dropout(0.2), 
    layers.Dense(1,kernel_initializer='normal',activation='relu') 
    # final layer has only one unit (change it if you do multi-output classification)
  ])
  
  optimizer = tf.keras.optimizers.SGD(lr=lr_val, momentum=momentum_val, 
                                      decay=decay_val, nesterov=nesterov_val) # many other options for optmizer
  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error']) 
    #When dealing with classification, 'accuracy' is very useful as well
  return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
#Take 10 samples from training dataset for quick test
example_batch = test_dataset[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, 
                                           patience=100, mode='auto', baseline=None, 
                                           restore_best_weights=True)

## train the model
this takes a while, depending on the size of the dataset, the number of EPOCHS, and the settings of the early_stop function (patience, min_delta, etc.)

In [ ]:
EPOCHS = 1000 # nr of iteration for the network before stopping (ultimate stop...could be 5000, but takes a long time)

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, 
  validation_split=0.1, # split the data set in 90/10
  shuffle=True, verbose=2,
  callbacks=[early_stop])

## plot the learning curve/ the progress of training

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [Mean Conc. N]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  #plt.ylim([0,1])
  plt.legend()
  plt.savefig('mean_asb_error_lr' + str(lr_val) + '_moment' + str(momentum_val) + '_nest' + str(nesterov_val) + "_" + region + "_" + name + '.png', bbox_inches='tight')
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$(Mean Conc.)^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  #plt.ylim([0,3])
  plt.legend()
  plt.savefig('mean_sq_error_lr' + str(lr_val) + '_moment' + str(momentum_val) + '_nest' + str(nesterov_val) + "_" + region + "_" + name + '.png', bbox_inches='tight')
  # plt.show()

plot_history(history)

## run the model for prediction

In [ ]:
#Time for a real prediction on the test data set
print("predict the model on the test data set...")
f, (ax1,ax2) = plt.subplots(1,2, sharey=True)
test_predictions = model.predict(test_dataset).flatten()
r = r2_score(test_labels, test_predictions)
ax1.scatter(test_labels, test_predictions,alpha=0.5, label='$R^2$ = %.3f' % (r))
ax1.legend(loc="upper left")
ax1.set_xlabel('True Values [Mean Conc.]')
ax1.set_ylabel('Predictions [Mean Conc.]')
ax1.axis('equal')
ax1.axis('square')
ax1.set_xlim([0,1])
ax1.set_ylim([0,1])
_ = ax1.plot([-100, 100], [-100, 100], 'r:')
ax1.set_title('Test dataset')
f.set_figheight(30)
f.set_figwidth(10)

# Predict for the whole dataset
print("predict the model on the whole data set...")
dataset_labels = dataset.pop('agb')
dataset = dataset
dataset_predictions = model.predict(dataset).flatten()
r = r2_score(dataset_labels, dataset_predictions)
ax2.scatter(dataset_labels, dataset_predictions, alpha=0.5, label='$R^2$ = %.3f' % (r))
ax2.legend(loc="upper left")
ax2.set_xlabel('True Values [Mean Conc.]')
ax2.set_ylabel('Predictions [Mean Conc.]')
ax2.axis('equal')
ax2.axis('square')
ax2.set_xlim([0,1])
ax2.set_ylim([0,1])
_ = ax2.plot([-100, 100], [-100, 100], 'r:')
ax2.set_title('Whole dataset')
# plt.show()
plt.savefig('R_scaled_lr' + str(lr_val) + '_moment' + str(momentum_val) + '_nest' + str(nesterov_val) + "_" + region + "_" + name + '.png', bbox_inches='tight')
# plt.close('all')

In [ ]:
# Undo scale step
# Test data set
test_dataset['agb'] = test_predictions
inverse_data = sc.inverse_transform(test_dataset)
inverse_data = pd.DataFrame(inverse_data, columns = dataset_orig.columns)
test_predictions = inverse_data.pop('agb')
test_labels = test_dataset_orig.pop('agb')

# Whole dataset
dataset['agb'] = dataset_predictions
inverse_data = sc.inverse_transform(dataset)
inverse_data = pd.DataFrame(inverse_data, columns = dataset.columns)
dataset_predictions = inverse_data.pop('agb')
dataset_labels = dataset_orig.pop('agb')


In [ ]:
# Plot Rsquared - scatter plot

f, (ax1,ax2) = plt.subplots(1,2, sharey=True)

# Test dataset
r = r2_score(test_labels, test_predictions)
ax1.scatter(test_labels, test_predictions, alpha=0.5, label='$R^2$ = %.3f' % (r))
ax1.legend(loc="upper left")
ax1.set_xlabel('True Values [Mean Conc.]')
ax1.set_ylabel('Predictions [Mean Conc.]')
ax1.axis('equal')
ax1.axis('square')
ax1.set_xlim([0,max(test_labels.max(), test_predictions.max())])
ax1.set_ylim([0,max(test_labels.max(), test_predictions.max())])
_ = ax1.plot([-max(test_labels.max(), test_predictions.max()), max(test_labels.max(), test_predictions.max())], [-max(test_labels.max(), test_predictions.max()), max(test_labels.max(), test_predictions.max())], 'r:')
ax1.set_title('Test dataset')
f.set_figheight(30)
f.set_figwidth(10)
#plt.show()

# Whole dataset
r = r2_score(dataset_labels, dataset_predictions)
ax2.scatter(dataset_labels, dataset_predictions, alpha=0.5, label='$R^2$ = %.3f' % (r))
ax2.legend(loc="upper left")
ax2.set_xlabel('True Values [Mean Conc.]')
ax2.set_ylabel('Predictions [Mean Conc.]')
ax2.axis('equal')
ax2.axis('square')
ax2.set_xlim([0,max(dataset_labels.max(), dataset_predictions.max())])
ax2.set_ylim([0,max(dataset_labels.max(), dataset_predictions.max())])
_ = ax2.plot([-max(dataset_labels.max(), dataset_predictions.max()), max(dataset_labels.max(), dataset_predictions.max())], [-max(dataset_labels.max(), dataset_predictions.max()), max(dataset_labels.max(), dataset_predictions.max())], 'r:')
ax2.set_title('Whole dataset')
# plt.show()
plt.savefig('R_unscaled_lr' + str(lr_val) + '_moment' + str(momentum_val) + '_nest' + str(nesterov_val) + "_" + region + "_" + name + '.png', bbox_inches='tight')

## Get the results

In [ ]:
AGB_results = pd.DataFrame({"Lat":inverse_data["Lat"], "Lon":inverse_data["Lat"], "AGB": dataset_predictions})
pos1 = dataset0.notna().all(axis = 1)
out_biomass = xr.zeros_like(biomass_2010)
out_biomass_vals = np.array(out_biomass[0,:,:])
out_biomass_vals_sub = out_biomass_vals[pos]
out_biomass_vals_sub[pos1] = AGB_results["AGB"]
out_biomass_vals_sub[pos1== False] = np.nan
out_biomass_vals[pos] = out_biomass_vals_sub
out_biomass_vals.shape
out_biomass[0,:,:] = out_biomass_vals

### Plot map of predicted biomass

In [ ]:
out_biomass[0,:,:].plot.pcolormesh(figsize=(16,10))
plt.title('Predicted AGB')
plt.ylabel('Lat')
plt.xlabel('Lon')
plt.savefig("predicted_biomass_2010_map_" + region + "_" + name + ".png",dpi = 300)
plt.show()

### plot difference between predicted and reference biomass

In [ ]:
biomass_diff = out_biomass - biomass_2010
biomass_diff = biomass_diff * (land_mask == 1) * (pos)
biomass_diff_vals = np.array(biomass_diff[0,:,:])
biomass_diff_sub = biomass_diff_vals[pos]
biomass_diff_sub[pos1==False] = 0
biomass_diff_vals[pos] = biomass_diff_sub
biomass_diff[0,:,:] = biomass_diff_vals

In [ ]:
biomass_diff[0,:,:].plot.pcolormesh(figsize=(16,10))
plt.title('Predicted - Reference AGB')
plt.ylabel('Lat')
plt.xlabel('Lon')
plt.savefig("diff_biomass_2010_map_" + region + "_" + name + ".png",dpi = 300)
plt.show()